# Double, Dueling DQN 구현


In [1]:
import gym
import numpy as np
import random
import tensorflow as tf
import tensorflow.contrib.slim as slim
import matplotlib.pyplot as plt
import scipy.misc
import os
%matplotlib inline

In [3]:
from gridworld import gameEnv

env = gameEnv(partial=False, size = 5)

AttributeError: module 'scipy.misc' has no attribute 'imresize'

In [5]:
class Qnetwork():
    def __init(self,h_size):
        # 네트워크는 게임으로부터 하나의 프레임을 받아 이를 배열로 인식 - flattening
        # 그 다음 배열의 크기를 재조절하고 4개의 합성곱 계층을 거쳐 처리한다.
        self.scalarInput = tf.placeholder(shape=[None,21168], dtype=tf.float32)
        self.imageIn = tf.reshape(self.scalarInput,shape=[-1,84,84,3])
        self.conv1 = slim.conv2d(\
                                inputs=self.imageIn, num_outputs=32, kernel_size = [8,8],stride=[4,4],\
                                padding='VALID', biases_initializer = None)
        self.conv2 = slim.conv2d(\
                                inputs=self.conv1,num_outputs=64,kernel_size=[4,4],stride=[2,2], \
                                padding='VALID', biases_initializer = None)
        self.conv3 = slim.conv2d(\
                                inputs=self.conv1,num_outputs=64,kernel_size=[3,3],stride=[1,1], \
                                padding='VALID', biases_initializer = None)
        self.conv4 = slim.conv2d(\
                                inputs=self.conv1,num_outputs=64,kernel_size=[7,7],stride=[1,1], \
                                padding='VALID', biases_initializer = None)
        
        # 마지막 합성곱 계층에서 출력값을 취한 후
        # 이를 어드밴티지 스트림과 가치 스트림으로 분리한다.
        self.streamAC,self.streamVC = tf.split(self.conv4, 2,3)
        self.streamA = slim.flatten(self.streamAC)
        self.streamV = slim.flatten(self.streamVC)
        self.AW = tf.Variable(tf.random_normal([h_size//2, env.actions]))
        self.VW = tf.Variable(tf.random_normal([h_size//2,1]))
        self.Advantage = tf.matmul(self.streamA,self.AW)
        self.Value = tf.matmul(self.streamV,self.VW)
        
        # 최종 Q 값을 얻기 위해 어드밴티지 스트림과 가치 스트림을 조합
        self.Qout = self.Value + tf.subtract(self.Advantage, \
                                            tf.reduce_mean(self.Advantage, axis=1,keep_dims=True))
        self.predict = tf.argmax(self.Qout,1)
        
        # 타깃 Q 값과 예측 Q 값의 차의 제곱합을 구함으로써 비용 계산
        self.targetQ = tf.placeholder(shape=[None],dtype=tf.float32)
        self.actions = tf.placeholder(shape=[None],dtype=tf.int32)
        self.actions_onehot = tf.one_hot(self.actions, env.actions, dtype=tf.float32)
        
        self.Q = tf.reduce_sum(tf.multiply(self.Qout, self.actions_onehot), axis = 1)
        
        self.td_error = tf.square(self.targetQ - self.Q)
        self.loss = tf.reduce_mean(self.td_error)
        self.trainer = tf.train.AdamOptimizer(learning_rate = 0.0001)
        self.updateModel = self.trainer.minimize(self.loss)
        

In [7]:
class experience_buffer():
    def __init__(self, buffer_size = 50000):
        self.buffer = []
        self.buffer_size = buffer_size
        
    def add(self, experience):
        if len(self.buffer) + len(experience) >= self.buffer_size:
            self.buffer[0:(len(experience) + len(self.buffer)) - self.buffer_size] = []
        self.buffer.extend(experience)
        
    def sample(self, size):
        return np.reshape(np.array(random.sample(self.buffer,size), [size,5]))


In [8]:
def processState(states):
    return np.reshape(states,[21168])


In [10]:
def updateTargetGraph(tfVars, tau):
    total_vars = len(tfVars)
    op_holder = []
    for idx, var in enumerate(tfVars[0:total_vars//2]):
        op_holder.append(tfVars[idx+total_vars//2].assign((var.value()*tau) + \
                                                         ((1-tau)*tfVars[idx+total_vars//2].value())))
    return op_holder

def updateTarget(op_holder, sess):
    for op in op_holder:
        sess.run(op)

In [11]:
# parameters for learning

batch_size = 32 # 각 학습 단계에서 사용할 배치의 수
update_freq = 4 # 학습 단계 업데이트 빈도
y = .99 # 타깃 Q 값에 대한 discount factor
starE = 1 # epsilon - 시작 시의 랜덤한 액션 확률값 : epsilon
endE = 0.1 # 최종 랜덤 액션 확률값
anneling_step = 10000 # starE에서 endE로 줄어드는 데에 필요한 학습 단계의 수
num_episodes = 10000 # 네트워크를 학습시키기 위한 게임 환경 에피소드의 수
pre_train_steps = 10000 # 학습 시작 전 랜덤 액션의 단계 수
max_epLength = 50 # 허용되는 최대 에피소드 길이
load_model = False # 저장된 모델을 로드할지 여부
path = "./dqn" #모델을 저장할 경로
h_size = 512 #어드밴티지/가치 스트림으로 분리되기 전 마지막 합성곱 계층의 크기
tau = 0.001 # 타깃 네트워크를 제 1 네트워크로 업데이트하는 비율


In [19]:
tf.reset_default_graph()
mainQN = Qnetwork(h_size)
targetQN = Qnetwork(h_size)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

trainables = tf.trainable_variables()

targetOps = updateTargetGraph(trainables, tau)

myBuffer = experience_buffer()

# 랜덤 액션이 감소하는 비율을 설정
e = startE
stepDrop = (startE - endE) / anneling_steps

# 보상의 총계와  에피소드별 단계 수를 담을 리스트를 생성한다.
jList = []
rList = []
total_steps = 0

# 모델을 저장할 경로 생성.
if not os.path.exists(path):
    os.makedirs(path)

    with tf.Session() as sess:
        sess.run(init)
        if load_model == True:
            print('Loading Model...')
            ckpt = tr.train.get_checkpoint_state(path)
            saver.restore(sess,ckpt.model_checkpoint_path)
        # 타깃 네트워크가 제 1 네트워크와 동일하도록 설정
        updateTarget(targetOps,sess)
        for i in range(num_episodes):
            episodeBuffer = experience_buffer()
            # 환경을 리셋하고 첫 번째 새로운 관찰을 얻는다.
            s = env.reset()
            s = processState(s)
            d = False
            rAll = 0
            j = 0
            # Q network
            # 에피소드 길이 (50) 이내까지 수행
            while j < max_epLength:
                j += 1
                # Q 네트워크에서 (e의 랜덤 액션 선택 확률 적용) 그리디하게 액션 선택
                if np.random.rand(1) < e or total_steps < pre_train_steps:
                    a = np.random.randint(0,4)
                else:
                    a = sess.run(mainQN.predict,feed_dict ={mainQN.scalarInput:[s]})[0]
                s1,r,d = env.step(a)
                s1 = processState(s1)
                total_steps+=1
                # 에피소드 버퍼에 경험을 저장
                episodeBuffer.add(np.reshape(np.array([s,a,r,s1,d]),[1,5]))
                
                if total_steps > pre_train_steps:
                    if e > endE:
                        e -= stepDrop
                        
                    if total_step % (update_freq) == 0:
                        # 경험에서 랜덤하게 배치 하나를 샘플링
                        trainBatch = myBuffer.sample(batch_size)
                        # 타깃 Q 값에 대해 더블 DQN 업데이트를 수행
                        Q1 = sess.run(mainQN.predict, \
                                     feed_dict = {mainQN.scalarInput:np.vstack(trainBatch[:,3])})
                        Q2 = sess.run(targetQN.predict, \
                                     feed_dict = {targetQN.scalarInput:np.vstack(trainBatch[:,3])})
                        end_multiplier = -(trainBatch[:,4]-1)
                        doubleQ = Q2[range(batch_size),Q1]
                        targetQ = trainBatch[:,2] + (y*doubleQ * end_multiplier)
                        # 타깃 값을 이용해 네트워크를 업데이트
                        _ = sess.run(mainQN.updateModel, \
                                    feed_dict = {mainQN.scalarInput:np.vstack(trainBatch[:,0]), \
                                                mainQN.target:targetQ, mainQN.actions:trainBatch[:,1]})
                        
                        # 타깃 네트워크가 제 1 네트워크와 동일하도록 설정
                        updateTarget(targetOps,sess)
                rAll += r
                s = s1
                
                if d == True:
                    break
                
            myBuffer.add(episodeBuffer.buffer)
            jList.append(j)
            rList.append(rAll)
            # 정기적으로 모델 저장
            if i % 1000 == 0:
                saver.save(sess, path+ '/model-'+str(i)+'.cptk')
                print("Saved Model")
            if len(rList) % 10 == 0:
                print(total_steps,np.mean(rList[-10:], e)
        
            #saver.save(sess, path+ '/model-'+str(i)+'.cptk')
                      
    print("Percent of success episodes: " +  str(sum(rList)/num_episodes))

SyntaxError: invalid syntax (<ipython-input-19-9501cff38a65>, line 99)